In [ ]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt
import pylab

print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']

In [ ]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=18000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]

In [ ]:
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[ ]')
# _df0.head()

for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [ ]:

start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if letter.isalpha() or pattern.match(letter):
            exec('if \'{let}\' not in {lang}Alphabets.keys():\n\
                     {lang}Alphabets[letter] = {lang}Size\n\
                     {lang}Size += 1'.format(let=letter, lang=language))#, language, language, language))
            sentence = sentence + letter
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
for language in languages:
    exec('{lang}Model=np.zeros(shape=(({lang}Size+1),({lang}Size+1),({lang}Size+1)))'.format(lang=language))#, language, language, language))

In [ ]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            third = tweet[i+2]
            exec('firstIndex = %sAlphabets[first]'%(language))
            exec('secondIndex = %sAlphabets[second]'%(language))
            exec('thirdIndex = %sAlphabets[third]'%(language))
            exec('%sModel[firstIndex][secondIndex][thirdIndex] += 1'%(language))
            
for language in languages:
    exec('for x in range({lang}Model.shape[0]):\n\
    for y in range({lang}Model.shape[0]):\n\
        sumCol = 0\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] += delta\n\
            sumCol += {lang}Model[x][y][z]\n\
        for z in range({lang}Model.shape[0]):\n\
            {lang}Model[x][y][z] /= sumCol\n\
                \n\
{lang}Model = np.log10({lang}Model)'.format(lang=language))

print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
df = pd.read_csv('../OriginalDataSet/test-tweets-given.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=100)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['TweetID', 'Language', 'Tweet']].copy()
# pattern = re.compile('[a-z ]')
# _df0.head()

In [ ]:
probability = {}
for index, row in _df0.iterrows():
    for language in languages:
        exec("%sProb=math.log10(1/6)" % (language))
    tweetID = row['TweetID']
    langTweet = row['Language']
    tweet = row['Tweet']
    
    for i in range(len(tweet)-2):
        first = tweet[i]
        second = tweet[i+1]
        third = tweet[i+2]
        for language in languages:
            exec('if ((first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys())):\n\
    prob = {lang}Model[-1][-1][-1]\n\
elif ( (first not in {lang}Alphabets.keys()) and (second not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][-1][index]\n\
elif ( (first not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[second]\n\
    prob = {lang}Model[-1][index][-1]\n\
elif ( (second not in {lang}Alphabets.keys()) and (third not in {lang}Alphabets.keys()) ):\n\
    index = {lang}Alphabets[first]\n\
    prob = {lang}Model[index][-1][-1]\n\
elif first not in {lang}Alphabets.keys():\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[-1][secondIndex][thirdIndex]\n\
elif second not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][-1][thirdIndex]\n\
elif third not in {lang}Alphabets.keys():\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    prob = {lang}Model[firstIndex][secondIndex][-1]\n\
else:\n\
    firstIndex = {lang}Alphabets[first]\n\
    secondIndex = {lang}Alphabets[second]\n\
    thirdIndex = {lang}Alphabets[third]\n\
    prob = {lang}Model[firstIndex][secondIndex][thirdIndex]\n\
{lang}Prob = prob + {lang}Prob\n'.format(lang='en'))
    for langu in languages:
        exec("probability['%s'] = %sProb"%(langu, langu))
    result = max(probability, key=probability.get)
    print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', langTweet, 'correct' if (langTweet==result) else 'wrong')

In [7]:
str = 'if self.vocab==\'2\':\n\
    if not first.isalpha():\n\
        prob = 0\n\
        self.check = false\n\
else:\n\
    if not self.pattern.match(first):\n\
        prob = 0\n\
        self.check=false\n\
if(self.check):\n\
    if (first not in self.{lang}Alphabets.keys()):\n\
        prob = self.{lang}Model[-1]\n\
    else:\n\
        index = self.{lang}Alphabets[first]\n\
        prob = self.{lang}Model[index]\n\
{lang}Prob = prob + {lang}Prob\n\
self.check = true\n'.format(lang='en')
print(str)

if self.vocab=='2':
    if not first.isalpha():
        prob = 0
        self.check = false
else:
    if not self.pattern.match(first):
        prob = 0
        self.check=false
if(self.check):
    if (first not in self.enAlphabets.keys()):
        prob = self.enModel[-1]
    else:
        index = self.enAlphabets[first]
        prob = self.enModel[index]
enProb = prob + enProb
self.check = true



In [17]:
atr = 'if self.vocab==\'2\':\n\
    if ( (not first.isalpha()) or (not second.isalpha()) ):\n\
        prob = 0\n\
        self.check = False\n\
else:\n\
    if ( (not self.pattern.match(first)) or (not self.pattern.match(second)) ):\n\
        prob = 0\n\
        self.check=False\n\
if(self.check):\n\
    if ( (first not in self.{lang}Alphabets.keys()) and (second not in self.{lang}Alphabets.keys()) ):\n\
        prob = self.{lang}Model[-1][-1]\n\
    elif (second not in self.{lang}Alphabets.keys()):\n\
        index = self.{lang}Alphabets[first]\n\
        prob = self.{lang}Model[index][-1]\n\
    elif (first not in self.{lang}Alphabets.keys()):\n\
        index = self.{lang}Alphabets[second]\n\
        prob = self.{lang}Model[-1][index]\n\
    else:\n\
        firstIndex = self.{lang}Alphabets[first]\n\
        secondIndex = self.{lang}Alphabets[second]\n\
        prob = self.{lang}Model[firstIndex][secondIndex]\n\
{lang}Prob = prob + {lang}Prob\n\
self.check = True\n'.format(lang='en')
print(atr)

if self.vocab=='2':
    if ( (not first.isalpha()) or (not second.isalpha()) ):
        prob = 0
        self.check = False
else:
    if ( (not self.pattern.match(first)) or (not self.pattern.match(second)) ):
        prob = 0
        self.check=False
if(self.check):
    if ( (first not in self.enAlphabets.keys()) and (second not in self.enAlphabets.keys()) ):
        prob = self.enModel[-1][-1]
    elif (second not in self.enAlphabets.keys()):
        index = self.enAlphabets[first]
        prob = self.enModel[index][-1]
    elif (first not in self.enAlphabets.keys()):
        index = self.enAlphabets[second]
        prob = self.enModel[-1][index]
    else:
        firstIndex = self.enAlphabets[first]
        secondIndex = self.enAlphabets[second]
        prob = self.enModel[firstIndex][secondIndex]
enProb = prob + enProb
self.check = True



In [20]:
atr = 'if self.vocab==\'2\':\n\
    if ( (not first.isalpha()) or (not second.isalpha()) or (not third.isalpha()) ):\n\
        prob = 0\n\
        self.check = False\n\
else:\n\
    if ( (not self.pattern.match(first)) or (not self.pattern.match(second)) or (not self.pattern.match(third)) ):\n\
        prob = 0\n\
        self.check=False\n\
if(self.check):\n\
    if ((first not in self.{lang}Alphabets.keys()) and (second not in self.{lang}Alphabets.keys()) and (third not in self.{lang}Alphabets.keys())):\n\
        prob = self.{lang}Model[-1][-1][-1]\n\
    elif ( (first not in self.{lang}Alphabets.keys()) and (second not in self.{lang}Alphabets.keys()) ):\n\
        index = self.{lang}Alphabets[third]\n\
        prob = self.{lang}Model[-1][-1][index]\n\
    elif ( (first not in self.{lang}Alphabets.keys()) and (third not in self.{lang}Alphabets.keys()) ):\n\
        index = self.{lang}Alphabets[second]\n\
        prob = self.{lang}Model[-1][index][-1]\n\
    elif ( (second not in self.{lang}Alphabets.keys()) and (third not in self.{lang}Alphabets.keys()) ):\n\
        index = self.{lang}Alphabets[first]\n\
        prob = self.{lang}Model[index][-1][-1]\n\
    elif first not in self.{lang}Alphabets.keys():\n\
        secondIndex = self.{lang}Alphabets[second]\n\
        thirdIndex = self.{lang}Alphabets[third]\n\
        prob = self.{lang}Model[-1][secondIndex][thirdIndex]\n\
    elif second not in self.{lang}Alphabets.keys():\n\
        firstIndex = self.{lang}Alphabets[first]\n\
        thirdIndex = self.{lang}Alphabets[third]\n\
        prob = self.{lang}Model[firstIndex][-1][thirdIndex]\n\
    elif third not in self.{lang}Alphabets.keys():\n\
        firstIndex = self.{lang}Alphabets[first]\n\
        secondIndex = self.{lang}Alphabets[second]\n\
        prob = self.{lang}Model[firstIndex][secondIndex][-1]\n\
    else:\n\
        firstIndex = self.{lang}Alphabets[first]\n\
        secondIndex = self.{lang}Alphabets[second]\n\
        thirdIndex = self.{lang}Alphabets[third]\n\
        prob = self.{lang}Model[firstIndex][secondIndex][thirdIndex]\n\
        prob = self.{lang}Model[firstIndex][secondIndex][thirdIndex]\n\
        prob = self.{lang}Model[firstIndex][secondIndex][thirdIndex]\n\
{lang}Prob = prob + {lang}Prob\n\
self.check = True\n'.format(lang='en')
print(atr)

if self.vocab=='2':
    if ( (not first.isalpha()) or (not second.isalpha()) or (not third.isalpha()) ):
        prob = 0
        self.check = False
else:
    if ( (not self.pattern.match(first)) or (not self.pattern.match(second)) or (not self.pattern.match(third)) ):
        prob = 0
        self.check=False
if(self.check):
    if ((first not in self.enAlphabets.keys()) and (second not in self.enAlphabets.keys()) and (third not in self.enAlphabets.keys())):
        prob = self.enModel[-1][-1][-1]
    elif ( (first not in self.enAlphabets.keys()) and (second not in self.enAlphabets.keys()) ):
        index = self.enAlphabets[third]
        prob = self.enModel[-1][-1][index]
    elif ( (first not in self.enAlphabets.keys()) and (third not in self.enAlphabets.keys()) ):
        index = self.enAlphabets[second]
        prob = self.enModel[-1][index][-1]
    elif ( (second not in self.enAlphabets.keys()) and (third not in self.enAlphabets.keys()) ):
        index = self.enAlphabets[first]